# Validate Pre-Trained DL Models

This notebook loads the saved-state for all 4 models and validates them:

- main model (CNN + D2V)
- cnn-only model
- d2v-only model
- cnn-attention model


In [ ]:
from gensim.models.keyedvectors import KeyedVectors
from gensim.parsing.preprocessing import preprocess_string
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import os
import math
import csv
import pickle
import time
import bz2

In [ ]:
# Set seed for random generators

seed = 24
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
# Set the device type as cpu or cuda depending upon the execution environment.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# Clone git repo containing all the pre-processed data files. THIS IS ONLY NEEDED IN CLOUD ENVIRONMENT.

!git clone https://github.com/manuv3/cs598-dl-project.git

In [ ]:
DATA_DIR = './cs598-dl-project/data'

# USE BELOW FOR LOCAL TESTING
# DATA_DIR = '../data'

In [ ]:
'''
Import the "codes" DataFrame, with HADM_ID (Hospitalization ID) as index and multi-hot encoding of 
ICD9-codes (booleans) as columns. Its dimensions are a 52691 rows × 6984 columns. So, we have 6984 ICD9 codes.
'''

codes = pd.read_pickle(DATA_DIR + '/diagnoses.pkl.gz')
codes

In [ ]:
'''
Split the HADM_IDs (hospitalization IDs) into train-test in 90:10 ratio. Two lists are generated:
    - hadm_ids_train
    - hadm_ids_test
'''

hadm_ids_train, hadm_ids_test = train_test_split(codes.index.tolist(), test_size = 0.10, random_state=seed)
print(hadm_ids_train)

In [ ]:
'''
Load the Doc2Vec embeddings for discharge summary reports, generated during pre-processing step. The data is in 
Gensim KeyedVector format.
'''

dv = KeyedVectors.load(DATA_DIR + '/dv.kv')

In [ ]:
'''
Load the Word2Vec embeddings of all the words in vocabulary of the whole corpus, generated during the 
pre-processing step. This data is in Gensim KeyedVector format.
'''

wv = KeyedVectors.load(DATA_DIR + '/wv.kv')

In [ ]:
'''
Load the dictionary mapping HADM_ID with a tokenized document. These tokens are basically the list of words 
belonging to the corresponding Discharge summary report.
'''

with bz2.open(DATA_DIR + '/tokens_map.pkl.bz2', 'rb') as handle:
  tokens_dict = pickle.load(handle)

In [ ]:
# how many samples per batch to load
batch_size = 50

In [ ]:
'''
Define the implementation of PyTorch Dataset. This is very light-weight, as all the data tensors (dv_train, 
dv_test, tokens_train, tokens_test, and codes_train) have already been pushed to GPU memory. So they can be 
referenced by HADM_ID index. This dataset simply returns the input index as the data in __getitem()__ method, which 
will be used during model training to access the input and output data from data tensors.
'''

class DocumentsDataset(Dataset):
    def __init__(self, hadm_ids):
        super(DocumentsDataset).__init__()
        self.hadm_ids = hadm_ids
    def __len__(self):
        return len(self.hadm_ids)
    def __getitem__(self, idx):
        hadm_id = self.hadm_ids[idx]
        tokens = tokens_dict[hadm_id]
        word_vecs = torch.Tensor(wv.__getitem__(tokens))
        x_cnn = torch.zeros((700, 100))
        x_cnn[0:word_vecs.shape[0]] = word_vecs
        x_d2v = torch.Tensor(dv[str(hadm_id)].tolist())
        y = torch.tensor(codes.loc[hadm_id].to_numpy())
        return x_d2v, x_cnn, y 

In [ ]:
# prepare test dataloader
test_loader = DataLoader(DocumentsDataset(hadm_ids_test), batch_size = batch_size)

print("# of val batches:", len(test_loader))

In [ ]:
'''
Define the model class which represents the D2V sub-component that uses single fully connected layer to fine-tune 
the Doc2Vec embeddings. This representation is passed through a Dropout layer to perform regularization, and the output is the 
vector representing the input document.
'''

class D2V(nn.Module):
    def __init__(self, dropout = 0.20):
        super(D2V, self).__init__()
        self.fc = nn.Linear(128, 64)
        self.dropout = nn.Dropout(p = dropout)
    def forward(self, x):
        y = self.dropout(F.relu(self.fc(x)))
        return y

In [ ]:
'''
Define the model class which is the building block of the CNN sub-component containining a single layer of 
multi-channel CNN kernel, activation function, and max-pooling layer.
'''

class CNN(nn.Module):
    def __init__(self, kernel_size, in_channels = 1, out_channels = 64):
        super(CNN, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, (kernel_size, 100))
        self.pool = nn.MaxPool2d((700 - kernel_size + 1,1))
    def forward(self, x):
        y = self.pool(F.relu(self.conv(x))).squeeze()
        return y

In [ ]:
'''
Define the model class for CNN sub-component. This component concatenates the output of 3 CNN components (described 
previously), each corresponding to different kernel size (3, 4, and 5 words). Its input is "concatenated" Word2Vec 
embeddings of all words within a document, which it passes to the three CNN components parallely, and then combines 
their output. This representation is passed through a Dropout layer to perform regularization, and the output is the 
vector representing the input document.
'''

class CNN_COMBINED(nn.Module):
    def __init__(self, dropout = 0.20):
        super(CNN_COMBINED, self).__init__()
        self.conv_3 = CNN(3)
        self.conv_4 = CNN(4)
        self.conv_5 = CNN(5)
        self.dropout = nn.Dropout(p = dropout)
    def forward(self, x):
        x = x.unsqueeze(1)
        return self.dropout(torch.cat((self.conv_3(x), self.conv_4(x), self.conv_5(x)), dim = 1))

In [ ]:
'''
Define the class for the main model, which uses both from D2V and CNN_COMBINED components, in the Encoder layer.
'''

class Net(nn.Module):
    def __init__(self, dropout_d2v, dropout_cnn):
        super(Net, self).__init__()
        self.d2v = D2V(dropout = dropout_d2v)
        self.cnn = CNN_COMBINED(dropout = dropout_cnn)
        self.fc2 = nn.Linear(256, 6984)

    def forward(self, x_d2v, x_cnn):
        y_d2v = self.d2v(x_d2v)
        y_cnn = self.cnn(x_cnn)
        y = torch.sigmoid(self.fc2(torch.cat((y_d2v, y_cnn), dim = 1)))
        return y

In [ ]:
'''
Define the class for the cnn-only ablation model, which uses only the CNN_COMBINED component in the Encoder layer. 
'''

class Net_CNN(nn.Module):
    def __init__(self, dropout):
        super(Net_CNN, self).__init__()
        self.cnn = CNN_COMBINED(dropout)
        self.fc2 = nn.Linear(192, 6984)

    def forward(self, x):
        return torch.sigmoid(self.fc2(self.cnn(x)))

In [ ]:
'''
Define the class for the d2v-only ablation model, which uses only the D2V component in the Encoder layer. 
'''

class Net_D2V(nn.Module):
    def __init__(self, dropout):
        super(Net_D2V, self).__init__()
        self.d2v = D2V(dropout = dropout)
        self.fc = nn.Linear(64, 6984)

    def forward(self, x):
        return torch.sigmoid(self.fc(self.d2v(x)))

In [ ]:
'''
Define the class for the cnn + attention model.
'''

class Net_CNN_Attention(nn.Module):
    def __init__(self, kernel_size, dropout):
        super(Net_CNN_Attention, self).__init__()
        self.conv = nn.Conv1d(100, 50, kernel_size)
        self.att = nn.Linear(50, 6984, bias = False)
        self.dropout = nn.Dropout(p = dropout)
        self.fc = nn.Linear(50, 1)
    def forward(self, x):
        x = x.permute((0, 2, 1))
        y = torch.relu(self.conv(x))
        # apply attention
        alpha = F.softmax(self.att.weight.matmul(y), dim=2)
        # compute output by matrix multiplication of attention matrix with internal state
        y = alpha.matmul(y.permute(0, 2, 1))
        y = self.dropout(y)
        y = torch.sigmoid(self.fc(y)).squeeze()
        return y

In [ ]:
# Model Evaluation

from sklearn.metrics import *


def classification_metrics(Y_pred, Y_true):
    """
    Calculate peformance metrics using scikit-learn.
    
    Arguments:
        Y_pred: Long dtype Tensor of output values for the test set batch, as predicted by model.
        Y_true: Long dtype Tensor of true values in the test-set batch.
        
    Outputs:
        precision: overall micro-averaged precision score
        recall: overall micro-averaged recall score
        f1: overall micro-averaged f1 score
        
    REFERENCE: checkout https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
"""
    
    precision, recall, f1score = precision_score(Y_true, Y_pred, average = 'micro'), \
                                           recall_score(Y_true, Y_pred, average = 'micro'), \
                                           f1_score(Y_true, Y_pred, average = 'micro')
    return precision, recall, f1score


def evaluate(model, loader, threshold, only_cnn = False, only_d2v = False):
    """
    Evaluate the model.
    
    Arguments:
        model: Trained model of type nn.Module
        loader: Test DataLoader
        
    Outputs:
        precision: overall micro-averaged precision score
        recall: overall micro-averaged recall score
        f1: overall micro-averaged f1 score
    """
    
    model.eval()
    all_y_true = torch.LongTensor()
    all_y_pred = torch.LongTensor()
    for x_d2v,x_cnn, y in loader:
        if not only_cnn:
          x_d2v = x_d2v.to(device)
        if not only_d2v:
          x_cnn = x_cnn.to(device)
        if only_d2v:
          y_hat = model(x_d2v)
        elif only_cnn:
          y_hat = model(x_cnn)
        else:
          y_hat = model(x_d2v, x_cnn)
        y_pred = y_hat.detach().to('cpu').apply_(lambda x: 1 if x > threshold else 0)
        all_y_true = torch.cat((all_y_true, y.long()), dim=0)
        all_y_pred = torch.cat((all_y_pred,  y_pred.to('cpu').long()), dim=0)
        
    precision, recall, f1 = classification_metrics(all_y_pred.detach().numpy(), all_y_true.detach().numpy())
    print(f"precision: {precision:.3f}, recall: {recall:.3f}, f1: {f1:.3f}")
    return precision, recall, f1

In [ ]:
# Utility method to load pre-trained model state

    
def load_checkpoint(model, checkpoint_file):
    model.load_state_dict(torch.load('./cs598-dl-project/trained_models/' + checkpoint_file))

## Evaluate pre-trained main model

### Hyperparameters

|Hyperparameter|Value|
|:-------------|:----| 
|CLASSIFICATION_THRESHOLD|0.20|
|DROPOUT_D2V|0.30|
|DROPOUT_CNN|0.20|

    This model was trained for 20 epochs. 
    The final model was selected with state from 19th epoch.
    Time spent in training and evaluation: 1236.32 seconds.

### Results

After the end of run, the results should be similar to below table:

|Precision|Recall|F1-score|
|:--------|:-----|:-------|
|0.502|0.351|0.413|

In [ ]:
'''
Evaluate pre-trained main model
'''

# Hyperparameters
CLASSIFICATION_THRESHOLD = 0.20
DROPOUT_D2V = 0.30
DROPOUT_CNN = 0.20

# Define the model
main_model = Net(DROPOUT_D2V, DROPOUT_CNN)

# Load model state
load_checkpoint(main_model, 'main_model.pth')

# Move to GPU
main_model.to(device)

# Evaluate
precision, recall, f1 = evaluate(main_model, test_loader, threshold = CLASSIFICATION_THRESHOLD)

# Assert
assert (round(precision, 3), round(recall, 3), round(f1, 3)) == (0.502, 0.351, 0.413)


## Evaluate pre-trained cnn-only model (ablation)

### Hyperparameters

|Hyperparameter|Value|
|:-------------|:----| 
|CLASSIFICATION_THRESHOLD|0.20|
|DROPOUT_CNN|0.20|

    This model was trained for 15 epochs. 
    The final model was selected with state from 13th epoch.
    Time spent in training and evaluation: 1129.95 seconds.
        
### Results

After the end of run, the results should be similar to below table:

|Precision|Recall|F1-score|
|:--------|:-----|:-------|
|0.478|0.345|0.401|

In [ ]:
'''
Evaluate pre-trained cnn-only model
'''

# Hyperparameters
CLASSIFICATION_THRESHOLD = 0.20
DROPOUT_CNN = 0.20

# Define the model
cnn_only_model = Net_CNN(DROPOUT_CNN)

# Load model state
load_checkpoint(cnn_only_model, 'cnn_only_model.pth')

# Move to GPU
cnn_only_model.to(device)


# Evaluate
precision, recall, f1 = evaluate(cnn_only_model, test_loader, threshold = CLASSIFICATION_THRESHOLD, only_cnn = True)

# Assert
assert (round(precision, 3), round(recall, 3), round(f1, 3)) == (0.478, 0.345, 0.401)

## Evaluate pre-trained d2v-only model (ablation)

### Hyperparameters

|Hyperparameter|Value|
|:-------------|:----| 
|CLASSIFICATION_THRESHOLD|0.18|
|DROPOUT_D2V|0.10|

    This model was trained for 20 epochs. 
    The final model was selected with state from 15th epoch.
    Time spent in training and evaluation: 1188.31 seconds.
        
### Results

After the end of run, the results should be similar to below table:

|Precision|Recall|F1-score|
|:--------|:-----|:-------|
|0.350|0.261|0.299|

In [ ]:
'''
Evaluate pre-trained d2v-only model
'''

# Hyperparameters
CLASSIFICATION_THRESHOLD = 0.18
DROPOUT_D2V = 0.10

# Define the model
d2v_only_model = Net_D2V(DROPOUT_D2V)

# Load model state
load_checkpoint(d2v_only_model, 'd2v_only_model.pth')

# Move to GPU
d2v_only_model.to(device)


# Evaluate
precision, recall, f1 = evaluate(d2v_only_model, test_loader, threshold = CLASSIFICATION_THRESHOLD, only_d2v = True)

# Assert
assert (round(precision, 3), round(recall, 3), round(f1, 3)) == (0.350, 0.261, 0.299)

## Evaluate pre-trained cnn-attention model (additional model)

### Hyperparameters

|Hyperparameter|Value|
|:-------------|:----|
|CLASSIFICATION_THRESHOLD|0.30|
|DROPOUT_CNN|0.10|
|KERNEL_SIZE|5|

    This model was trained for 20 epochs. 
    The final model was selected with state from 19th epoch.
    Time spent in training and evaluation: 2240.29 seconds.
        
### Results

After the end of run, the results should be similar to below table:

|Precision|Recall|F1-score|
|:--------|:-----|:-------|
|0.520|0.372|0.434|

In [ ]:
'''
Evaluate pre-trained cnn-attention model
'''

# Hyperparameters
CLASSIFICATION_THRESHOLD = 0.30
DROPOUT_CNN = 0.10
KERNEL_SIZE = 5

# Define the model
cnn_attention_model = Net_CNN_Attention(kernel_size = KERNEL_SIZE, dropout = DROPOUT_CNN)

# Load model state
load_checkpoint(cnn_attention_model, 'cnn_attention_model.pth')

# Move to GPU
cnn_attention_model.to(device)


# Evaluate
precision, recall, f1 = evaluate(cnn_attention_model, test_loader, threshold = CLASSIFICATION_THRESHOLD, only_cnn = True)

# Assert
assert (round(precision, 3), round(recall, 3), round(f1, 3)) == (0.520, 0.372, 0.434)